In [1]:
from azureml.core import Model

In [2]:
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core import Workspace
svc_pr_password = "sWR5y]ocjTTMxT@7H13YXWFRcss=.nrN"

svc_pr = ServicePrincipalAuthentication(
    tenant_id="93f33571-550f-43cf-b09f-cd331338d086",
    service_principal_id="0a89e69e-7aae-4a80-8b76-7dc9f65c3d16",
    service_principal_password=svc_pr_password)


ws = Workspace(
    subscription_id="047ae087-7d35-4c57-8fe9-7a442cc9cf16",
    resource_group="Speech_Analytics",
    workspace_name="saab_ml_workspace",
    auth=svc_pr
    )

print("Found workspace {} at location {}".format(ws.name, ws.location))

Found workspace saab_ml_workspace at location westus2


In [2]:
##from azureml.core import Workspace
##ws = Workspace.from_config(path=".\config.json")

In [18]:
topic_model = Model.register(workspace=ws,
                       model_name="topic_model",
                       model_path="Topic_Modelling.pkl", # local path
                       description="A_topic_model")

Registering model topic_model


In [5]:
topic_prediction_model = Model.register(workspace=ws,
                       model_name="topic_prediction_model",
                       model_path="Topic_Prediction.pkl", # local path
                       description="A_topic_prediction_model")

Registering model topic_prediction_model


In [6]:
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("gensim")
myenv.add_conda_package("pyodbc")
myenv.add_conda_package("gensim")
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("time")

# Save the environment config as a .yml file
env_file = 'env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

Saved dependency info in env.yml


In [7]:
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(runtime= "python",
                                              entry_script="score.py",
                                              conda_file="env.yml")

In [8]:
from azureml.core.compute import ComputeTarget, AksCompute

cluster_name = 'aks-cluster-run'
compute_config = AksCompute.provisioning_configuration(location='westus')
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)

Creating..............................................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"


In [5]:
from azureml.core.webservice import AksWebservice,AciWebservice

classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 3.5)

In [6]:
from azureml.core.model import Model

topic_model = ws.models['topic_model']
#topic_prediction_model = ws.models['topic_prediction_model']
service = Model.deploy(workspace=ws,
                       name = 'topic-model-service_v2',
                       models = [topic_model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)

NameError: name 'production_cluster' is not defined

In [29]:
print(service.get_logs())

2020-02-19T06:52:49,791146660+00:00 - gunicorn/run 
/bin/bash: /azureml-envs/azureml_b577801dabfe2a395022a8230bbcddea/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_b577801dabfe2a395022a8230bbcddea/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_b577801dabfe2a395022a8230bbcddea/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2020-02-19T06:52:49,793510969+00:00 - iot-server/run 
2020-02-19T06:52:49,795233776+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_b577801dabfe2a395022a8230bbcddea/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b577801dabfe2a395022a8230bbcddea/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b577801dabfe2a395022a8230bbcddea/lib/libssl.so.1.0.0: no versi

In [18]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_70a2c3d7a8499f813c46b5380ccf2421/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_70a2c3d7a8499f813c46b5380ccf2421/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_70a2c3d7a8499f813c46b5380ccf2421/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_70a2c3d7a8499f813c46b5380ccf2421/lib/libtinfo.so.5: no version information available (required by /bin/bash)
bash: /azureml-envs/azureml_70a2c3d7a8499f813c46b5380ccf2421/lib/libtinfo.so.5: no version information available (required by bash)
2020-02-18T17:16:05,929669532+00:00 - rsyslog/run 
2020-02-18T17:16:05,929741132+00:00 - gunicorn/run 
2020-02-18T17:16:05,931304633+00:00 - iot-server/run 
2020-02-18T17:16:05,932393334+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_70a2c3d7a8499f813c46b5380ccf2421/lib/libcrypto.so.1.0.0